## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/'
fileName = 'jet.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
# Wczytaj obraz jet.bmp i wylicz jego histogram.

In [ ]:
J = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)
hist = cv2.calcHist([J],[0],None,[256],[0, 256])

figLena, axsLena = plt.subplots(1, 2)
figLena.set_size_inches(10, 5)
axsLena[0].imshow(J, 'gray', vmin=0, vmax=256)
axsLena[0].axis('off')
axsLena[1].plot(hist)
axsLena[1].grid()


In [ ]:
# W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (lm).

In [ ]:
J[:,:]

In [ ]:
# Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany. 
# Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego. W tym celu warto stworzyć tablicę, 
#zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli. Następnie znaleźć minimum - np.argmin.

In [ ]:
J_mean = int(J.mean())
J_mean

In [ ]:
# Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy H1 i H2. Dla każdego z nich wyliczyć histogram 
# skumulowany ( C1  i  C2 ) i wykonać normalizację. Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.

In [ ]:
H1 = hist[:J_mean]
H2 = hist[J_mean:256]

C1 = np.cumsum(H1)
C2 = np.cumsum(H2)

C1 = C1/np.max(C1)
C2 = C2/np.max(C2)

In [ ]:
#Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT. Należy tak przeskalować  C1  i  C2 , 
# aby uzyskać jednorodne przekształcenie. Tablicę  C1  wystarczy pomnożyć przez próg podziału. Tablicę  C2  należy przeskalować do 
# przedziału:  <lm+1;255> , gdzie  lm  jest progiem podziału.
#C1n=(lm)∗C1; 
#C2n=lm+1+(255−lm−1)∗C2;
#Następnie dwie części tablicy przekodowań należy połączyć.

In [ ]:
for x in range(len(C2)):
  C2[x] = J_mean + 1 + (255 - J_mean - 1) * C2[x]
  
C1n = J_mean * C1

C2n = C2.copy()
CC = np.concatenate((C1n,C2n))
print(CC)

In [ ]:
#Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu. Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
maxcc = np.max(CC)
CC = (CC/maxcc)*255
for x in range(len(CC)):
  CC[x] = int(CC[x])
plt.plot(CC)

In [ ]:
I = cv2.equalizeHist(J)
Heq = cv2.calcHist([J],[0],None,[256],[0, 256])
plt.plot(Heq)

In [ ]:
Jv2 = cv2.LUT(J,CC)
plt.imshow(J, 'gray', vmin=0, vmax=256)
plt.show()
plt.imshow(I, 'gray', vmin=0, vmax=256)
plt.show()
plt.imshow(Jv2, 'gray', vmin=0, vmax=256)